In [213]:
#import libraries
#The "TARGET" column is the variable to predict. It equals 0 for unsatisfied customers and 1 for satisfied customers.

#Models

from sklearn import decomposition
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier



#Transformation
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

#Graphics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Loading the train Data set
df_train = pd.read_csv("DataSet/train.csv")
df_test = pd.read_csv("DataSet/Test.csv")
Y_test = pd.read_csv("DataSet/sample_submission.csv")
df_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [3]:
#Basic statistics
df_train.describe()


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [4]:
#76020 variables and 371 observations. 
df_train.shape

(76020, 371)

In [5]:
#Check types for all collumns - Only Int and Float
dataTypeSeries = set(df_train.dtypes)
dataTypeSeries

{dtype('int64'), dtype('float64')}

In [6]:
#Checking the balance of the dataSET, as indicated, only 3% of my Dataset has the classification of satisfied client.

df=pd.DataFrame(df_train["ID"].groupby(df_train["TARGET"]).count())
Num_sat=int(df.iloc[1])

df_train["TARGET"].mean()


0.0395685345961589

In [7]:
#Moving ID to index, this way the column ID will be "removed" from the DataFrame
df_train.index=df_train["ID"]
df_train.drop("ID", axis=1, inplace=True)

In [8]:
df_train.shape

(76020, 370)

In [9]:
#Lista dos index com clientes satisfeitos e insatisfeitos
Index_Sat=np.array(df_train.loc[df_train.TARGET>0].index)
Index_InSat=np.array(df_train.loc[df_train.TARGET==0].index)
df_train.loc[Index_InSat].head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [10]:
#Gerando uma lista de itens randomicos com index das pessoas insatisfeitas para que temos as mesmas observações de 
#pessoas satisfeitas e insatisfeitas
random_Index_Inst=random.sample(list(Index_InSat),Num_sat)
df_train.loc[random_Index_Inst].head(5)

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
145303,2,31,0.0,3.0,75.15,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112962.72,0
27097,2,36,0.0,387.3,387.30,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31720.83,0
7005,2,33,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173771.40,0
12407,2,24,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,324032.76,0
84526,2,26,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,528859.59,0


In [11]:
#Now the Dataset has the same number of observations for 1 and 0.
List_dataset = [df_train.loc[Index_Sat],df_train.loc[random_Index_Inst]]
New_data_train = pd.concat(List_dataset)
New_data_train[["var3", "TARGET"]].groupby("TARGET").count()

,var3
TARGET,
0,3008
1,3008


In [12]:
# Now the median of observation for the classification is 50%
New_data_train["TARGET"].mean()

0.5

In [13]:
#Separado em dados em X, Y de treino e de dados de test
#Dataset Treino
Y_treino = New_data_train["TARGET"].values
New_data_train.drop("TARGET", axis=1, inplace=True)
X_treino = New_data_train
print(X_treino.shape, Y_treino.shape)


(6016, 369) (6016,)


In [14]:
#The test Dataset in this project was given with already splited in X and Y.
#Here we will drop the column ID

#Mudanças para X_Test
df_test.index = df_test["ID"]
df_test.drop("ID", axis=1, inplace=True)
X_test = df_test
#Mudanças para Y_test
Y_test.drop("ID", axis=1, inplace=True)
Y_test=Y_test["TARGET"].values


In [15]:
print(X_treino.shape, Y_treino.shape,X_test.shape, Y_test.shape)

(6016, 369) (6016,) (75818, 369) (75818,)


# Normalização dos dados

In [30]:
#Normalização com o normalizer

#Normalização de treino
x_treino_array=X_treino.values
scaler = Normalizer().fit(x_treino_array)
normalizedX = scaler.transform(x_treino_array)
X_treino_norm=pd.DataFrame(normalizedX,columns=X_treino.columns, index=X_treino.index)

#Normalização de test
x_test_array=X_test.values
scaler = Normalizer().fit(x_test_array)
normalizedX = scaler.transform(x_test_array)
X_test_norm=pd.DataFrame(normalizedX,columns=X_test.columns, index=X_test.index)


In [132]:
print(X_treino_norm.shape,X_test_norm.shape)

(6016, 369) (75818, 369)


# Redução da dimensionalidade

In [34]:
#Utilizando o PCA para reduzir a dimensionalidade
pca = decomposition.PCA(n_components = 100, 
                        whiten = True, 
                        svd_solver = 'randomized')

In [35]:
# O PCA é um algoritmo de aprendizagem não supervisionada
pca.fit(X_treino_norm)

PCA(n_components=100, svd_solver='randomized', whiten=True)

In [36]:
# E então aplicamos o modelo PCA nos dados de treino e de teste
X_treino_pca = pca.transform(X_treino_norm)
X_test_pca = pca.transform(X_test_norm)

In [134]:
# Shape
print(X_treino_pca.shape)
print(X_test_pca.shape)

(6016, 100)
(75818, 100)


In [130]:
X_treino_pca1=pd.DataFrame(X_treino_pca)
X_treino_pca1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.250474,0.007340,0.036352,-0.139060,0.041349,0.010037,-0.019318,0.061279,-0.011458,0.008806,...,0.075341,0.289747,0.307896,0.356551,-0.078689,-0.064124,-0.075210,0.017194,-0.286573,0.142086
1,-0.238634,-0.005462,0.013808,0.070568,-0.002258,-0.057996,0.096255,0.058022,0.019571,0.559036,...,-0.098473,0.887541,-1.733811,-0.011810,0.004823,-0.506047,0.380708,-0.512689,0.884508,-0.120695
2,4.295805,0.249828,13.051044,-0.474262,-6.462128,1.514992,-1.172369,-2.054792,0.060537,-0.037332,...,0.000002,-0.000012,0.000047,0.000026,-0.000008,-0.000045,-0.000034,0.000021,0.000014,0.000275
3,-0.250474,0.007340,0.036344,-0.139082,0.041225,0.010174,-0.019182,0.061533,-0.011471,0.008817,...,-0.041230,0.197931,-0.027745,-0.193540,0.048374,0.122256,0.127428,-0.041080,0.069987,-0.153637
4,-0.250473,0.007340,0.036362,-0.139041,0.041483,0.009893,-0.019464,0.061012,-0.011445,0.008805,...,-0.064928,0.594999,0.144668,0.554283,-0.017467,0.047025,-0.081970,0.001065,-0.017045,0.106008


# Criação dos Modelos

SVM - Suport vetor Machine

In [95]:
#ML com SVM
# Cria o modelo
modelo_svm = svm.SVC(C = 5., gamma = 0.001)
# Treinamento do modelo
modelo_svm.fit(X_treino_pca, Y_treino)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=5.0, gamma=0.001)

In [96]:
#Previsões
previsoes = modelo_svm.predict(X_test_pca)

In [97]:
print(modelo_svm.score(X_test_pca, Y_test))

0.7445329605106966


In [210]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
# Criando o modelo
modelo_xg = XGBClassifier()

# Treinando o modelo
modelo_xg.fit(X_treino_pca, Y_treino)

# Pront do modelo
print(modelo_xg)

# Fazendo previsões
y_pred = modelo_xg.predict(X_test_pca)
previsoes = [round(value) for value in y_pred]

# Avaliando as previsões
accuracy = accuracy_score(Y_test, previsoes)
print("Acurácia: %.2f%%" % (accuracy * 100.0))

D:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:01:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Acurácia: 71.73%


# Tentativa de otimização do Modelo

In [206]:
Best_features=list(pd.Series(modelo_xg.feature_importances_).nlargest(90).index)


In [207]:
X_treino_pca_bf=pd.DataFrame(X_treino_pca,index=X_treino.index )
X_test_pca_bf=pd.DataFrame(X_test_pca,index=X_test.index )
X_test_pca_bf.iloc[:,Best_features]

,15,16,58,6,61,2,46,51,11,70,...,32,59,63,9,40,87,66,54,35,77
ID,,,,,,,,,,,,,,,,,,,,,
2,0.014801,0.064674,-0.004150,-0.021730,-1.761149,0.035039,-0.010826,-0.030826,0.124944,0.356602,...,0.004425,-0.306440,-0.050839,0.010834,0.009990,0.409896,0.104597,-0.012342,0.005313,0.047555
5,0.015006,0.065556,-0.004261,-0.020374,-1.636940,0.035742,-0.010858,-0.023222,0.124390,0.065484,...,0.004801,-0.259585,-0.034301,0.009912,0.009524,-1.849531,-0.435068,-0.002793,0.005342,1.394448
6,0.021062,0.065472,-0.002759,-0.023492,-0.676229,0.029209,-0.017410,0.040542,0.060332,0.492246,...,0.038686,-0.322664,0.006727,0.055474,-0.010282,0.643709,-0.261641,-0.033975,0.009777,-0.177967
7,0.015209,0.066171,0.002917,-0.019556,0.746551,0.036369,-0.011620,-0.021890,0.124112,-0.107747,...,0.004931,0.237230,0.034402,0.008800,0.010579,0.064633,-0.032418,-0.022152,0.005428,-0.081849
9,0.014134,0.061344,0.000081,-0.026400,-0.255015,0.032872,-0.009062,-0.047024,0.126660,0.198743,...,0.001812,0.035180,0.002557,0.013648,0.012037,0.227116,0.017718,-0.023247,0.005007,-0.053153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151831,0.015351,0.067092,0.002640,-0.019108,0.941259,0.036340,-0.021983,0.112148,0.124220,0.191240,...,0.004303,0.223292,0.040665,0.008831,0.010702,-2.019039,-0.624705,0.015155,0.005718,-0.743943
151832,0.015131,0.065839,0.001350,-0.019877,0.170086,0.036187,-0.009281,-0.050066,0.124176,-0.006866,...,0.004911,0.129641,0.017988,0.009078,0.010450,0.133459,-0.006209,-0.025857,0.005346,-0.034240
151833,0.013661,0.059405,0.002800,-0.029033,0.738226,0.031738,-0.011120,-0.020386,0.127729,0.053739,...,0.002628,0.231800,0.033966,0.015891,0.010653,0.083230,-0.028598,-0.021192,0.005113,-0.127617


In [208]:

# Criando o modelo
modelo_xg2 = XGBClassifier()

# Treinando o modelo
modelo_xg2.fit(X_treino_pca_bf.iloc[:,Best_features], Y_treino)

# Pront do modelo
print(modelo_xg2)

# Fazendo previsões
y_pred2 = modelo_xg2.predict(X_test_pca_bf.iloc[:,Best_features])
previsoes2 = [round(value) for value in y_pred2]

# Avaliando as previsões
accuracy2 = accuracy_score(Y_test, previsoes2)
print("Acurácia: %.2f%%" % (accuracy2 * 100.0))

D:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Acurácia: 71.48%


In [211]:
#ML com SVM
# Cria o modelo
modelo_svm = svm.SVC(C = 5., gamma = 0.001)
# Treinamento do modelo
modelo_svm.fit(X_treino_pca_bf.iloc[:,Best_features], Y_treino)
#Previsões
previsoes = modelo_svm.predict(X_test_pca_bf.iloc[:,Best_features])
print(modelo_svm.score(X_test_pca_bf.iloc[:,Best_features], Y_test))

0.7443746867498483


In [ ]:
# Criando o modelo
modelo_xg2 = XGBClassifier()

# Treinando o modelo
modelo_xg2.fit(X_treino_pca_bf.iloc[:,Best_features], Y_treino)

# Pront do modelo
print(modelo_xg2)

# Fazendo previsões
y_pred2 = modelo_xg2.predict(X_test_pca_bf.iloc[:,Best_features])
previsoes2 = [round(value) for value in y_pred2]

# Avaliando as previsões
accuracy2 = accuracy_score(Y_test, previsoes2)
print("Acurácia: %.2f%%" % (accuracy2 * 100.0))